In [49]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import gym
import random
import statistics

In [47]:
LR = 1e-3
env= gym.make('CartPole-v0')
env.reset()
goal_steps = 500
score_requirement= 50
initial_games=10000

def random_games():
    for episode in range(5):
        env.reset()
        for t in range(200):
            env.render()
            action= env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                break
                
# random_games();

def init_population():
    training_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_games):
        score = 0;
        game_memory = []
        prev_obs = []
        
#         run game
        for _ in range(goal_steps):
            action= random.randrange(0,2)
            observation, reward, done, info = env.step(action)
            if len(prev_obs) > 0:
                game_memory.append([prev_obs,action])
            
            prev_obs = observation
            score+= reward
            if done:
                break
        
        if(score>= score_requirement):
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                     
                training_data.append([data[0],output])
        env.reset()
        scores.append(score)

    training_data_save = np.array(training_data)

    print('Average accepted score:',statistics.mean(accepted_scores))
    print('Median accepted score:',statistics.median(accepted_scores))
    print('Number of accepted scores',len(accepted_scores))
    return training_data

# init_population();

def neural_model(input_size):
    model = Sequential(
        Dense(128 ,input_dim = (input_size)),
        Activation('relu'),
        Dropout(0.2),
        Dense(256),
        Activation('relu'),
        Dropout(0.2),
        Dense(256),
        Activation('relu'),
        Dropout(0.2),
        Dense(256),
        Activation('relu'),
        Dropout(0.2),
        Dense(128),
        Activation('relu'),
        Dropout(0.2),
        Dense(2),
        Activation('softmax')
    )
    
    sgd = SGD(lr=LR, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              epochs=20,
              batch_size=128)
    score = model.evaluate(x_test, y_test, batch_size=128)
            
     

Average accepted score: 61.06481481481482
Median accepted score: 58.0
Number of accepted scores 432
